# Bitcoin Price Prediction with Prophet

This notebook uses Meta's Prophet library (free, no API key required) for Bitcoin price forecasting.

In [1]:
# Install required packages
!pip install prophet yfinance matplotlib -q

In [2]:
import yfinance as yf
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

print('Libraries loaded successfully!')

Libraries loaded successfully!


## 1. Download Bitcoin Historical Data

In [3]:
# Download Bitcoin data from Yahoo Finance
ticker = "BTC-USD"
btc_data = yf.download(ticker, period="2y")  # Last 2 years

# Prepare data for Prophet (needs 'ds' and 'y' columns)
df = btc_data.reset_index()[['Date', 'Close']].copy()
df.columns = ['ds', 'y']

print(f"Downloaded {len(df)} days of Bitcoin data")
print(df.tail())

Failed to get ticker 'BTC-USD' reason: Expecting value: line 1 column 1 (char 0)


[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['BTC-USD']: Exception('%ticker%: No price data found, symbol may be delisted (period=2y)')



Downloaded 0 days of Bitcoin data
Empty DataFrame
Columns: [ds, y]
Index: []


## 2. Train Prophet Model

In [4]:
# Initialize and train Prophet model
model = Prophet(
    daily_seasonality=True,
    yearly_seasonality=True,
    changepoint_prior_scale=0.05
)

model.fit(df)
print('Model trained successfully!')

ValueError: Dataframe has less than 2 non-NaN rows.

## 3. Make 30-Day Forecast

In [ ]:
# Create future dataframe for 30 days
future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)

# Display forecast
print("\nForecast for next 30 days:")
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(30))

## 4. Visualize Predictions

In [ ]:
# Plot forecast
fig1 = model.plot(forecast)
plt.title('Bitcoin Price Forecast (Prophet)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.show()

# Plot components (trend, seasonality)
fig2 = model.plot_components(forecast)
plt.show()

## 5. Generate Trading Signals

In [ ]:
# Simple strategy: Buy if predicted price increases
forecast_future = forecast.tail(30).copy()
forecast_future['predicted_signal'] = (forecast_future['yhat'].diff() > 0).astype(int)

# Save predictions
forecast_future[['ds', 'yhat', 'predicted_signal']].to_csv('bitcoin_predictions_prophet.csv', index=False)

print("\nTrading Signals (1=Buy, 0=Sell):")
print(forecast_future[['ds', 'yhat', 'predicted_signal']])
print("\nSaved to: bitcoin_predictions_prophet.csv")

## 6. Calculate Simple Statistics

In [ ]:
# Calculate statistics
current_price = df['y'].iloc[-1]
predicted_price_30d = forecast_future['yhat'].iloc[-1]
price_change = predicted_price_30d - current_price
price_change_pct = (price_change / current_price) * 100

print(f"\n=== 30-Day Forecast Summary ===")
print(f"Current Price: ${current_price:,.2f}")
print(f"Predicted Price (30 days): ${predicted_price_30d:,.2f}")
print(f"Expected Change: ${price_change:,.2f} ({price_change_pct:+.2f}%)")
print(f"\nBuy Signals: {forecast_future['predicted_signal'].sum()} days")
print(f"Sell Signals: {(1 - forecast_future['predicted_signal']).sum()} days")